In [ ]:
import concurrent.futures
from tqdm import tqdm
import xml.etree.ElementTree as ET
import os
import cv2
import pandas as pd
import json
import time
from typing import List, Tuple
from google.cloud import vision_v1
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = ''
client = vision_v1.ImageAnnotatorClient()

In [3]:
def google_cloud_vision_response(img_path):
    try:
        with open(img_path, 'rb') as img_file:
            content = img_file.read()
        imgocr = vision_v1.Image(content=content)
        image_context = vision_v1.ImageContext()
        with concurrent.futures.ThreadPoolExecutor() as executor:
            future = executor.submit(client.document_text_detection, image=imgocr, image_context=image_context)
            with tqdm(total=1, desc=f"Processing {img_path}") as pbar:
                while not future.done():
                    pbar.update(0.1)  # Arbitrary update to refresh the progress bar
            response = future.result()
        return response
    except Exception as e:
        print(f"Error in Google Cloud Vision API for {img_path}: {e}")
        return None

In [4]:
# def extract_bounding_boxes(xml_file):
#     # Parse the XML file
#     tree = ET.parse(xml_file)
#     root = tree.getroot()

#     # Initialize an empty list to store the bounding boxes
#     bounding_boxes = []

#     # Extract objects
#     objects = root.findall('object')

#     # Process objects in parallel
#     with concurrent.futures.ThreadPoolExecutor() as executor, tqdm(total=len(objects), desc="Processing Objects") as pbar:
#         # Define a function to process each object and extract its bounding box
#         def process_object(obj):
#             bndbox = obj.find('bndbox')
#             xmin = int(bndbox.find('xmin').text)
#             ymin = int(bndbox.find('ymin').text)
#             xmax = int(bndbox.find('xmax').text)
#             ymax = int(bndbox.find('ymax').text)
#             return (xmin, ymin, xmax, ymax)

#         # Submit each object processing task to the executor
#         future_to_obj = {executor.submit(process_object, obj): obj for obj in objects}
#         for future in concurrent.futures.as_completed(future_to_obj):
#             obj = future_to_obj[future]
#             try:
#                 result = future.result()
#                 bounding_boxes.append(result)
#             except Exception as e:
#                 print(f"Error processing object: {e}")
#             finally:
#                 pbar.update(1)

#     return bounding_boxes 

def extract_bounding_boxes(xml_file: str) -> List[Tuple[int, int, int, int]]:
    # Parse the XML file
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Get image dimensions from the XML
    size = root.find('size')
    img_width = int(size.find('width').text)
    img_height = int(size.find('height').text)

    # Initialize an empty list to store the bounding boxes
    bounding_boxes = []

    # Extract objects
    objects = root.findall('object')

    # Process objects in parallel
    with concurrent.futures.ThreadPoolExecutor() as executor, tqdm(total=len(objects), desc="Processing Objects") as pbar:
        # Define a function to process each object and extract its bounding box
        def process_object(obj):
            bndbox = obj.find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)
            return (xmin, ymin, xmax, ymax)

        # Submit each object processing task to the executor
        future_to_obj = {executor.submit(process_object, obj): obj for obj in objects}
        for future in concurrent.futures.as_completed(future_to_obj):
            obj = future_to_obj[future]
            try:
                result = future.result()
                bounding_boxes.append(result)
            except Exception as e:
                print(f"Error processing object: {e}")
            finally:
                pbar.update(1)

    # Helper functions for sorting
    def distance_from_top(bbox):
        return bbox[1]  # ymin

    def distance_from_left(bbox):
        return bbox[0]  # xmin

    # Separate and sort left side records
    left_records = sorted([bbox for bbox in bounding_boxes if bbox[0] < img_width / 2],
                          key=lambda x: (distance_from_top(x), distance_from_left(x)))

    # Separate and sort right side records
    right_records = sorted([bbox for bbox in bounding_boxes if bbox[0] >= img_width / 2],
                           key=lambda x: (distance_from_top(x), -distance_from_left(x)))

    # Merge both sequences
    ordered_bounding_boxes = left_records + right_records

    return ordered_bounding_boxes

In [5]:
def get_response_inside_roi(response, image_file, xml_file):
    if response is None:
        return None

    # Extract bounding boxes from the XML file
    bounding_boxes = extract_bounding_boxes(xml_file)

    # Initialize a list to store the texts from each bounding box
    ocr_results = []

    # Define a function to process each bounding box and extract text
    def process_bbox(bbox):
        xmin, ymin, xmax, ymax = bbox
        words = []

        # Iterate through each page, block, paragraph, and word in the OCR response
        for page in response.full_text_annotation.pages:
            for block in page.blocks:
                for paragraph in block.paragraphs:
                    for word in paragraph.words:
                        # Combine the symbols to form the full word text
                        word_text = ''.join([symbol.text for symbol in word.symbols])

                        # Extract the word's bounding box vertices
                        word_bounding_box = word.bounding_box.vertices
                        x1 = word_bounding_box[0].x
                        y1 = word_bounding_box[0].y
                        x2 = word_bounding_box[2].x
                        y2 = word_bounding_box[2].y

                        # Check if the word's bounding box is within the current bounding box
                        if (xmin <= x1 <= xmax and xmin <= x2 <= xmax and ymin <= y1 <= ymax and ymin <= y2 <= ymax):
                            words.append(word_text)

        # If there are words found within the current bounding box, combine them
        # and add them as a paragraph to the list
        if words:
            paragraph_text = ' '.join(words)
            return paragraph_text
        else:
            return None

    # Process bounding boxes in parallel
    with concurrent.futures.ThreadPoolExecutor() as executor, tqdm(total=len(bounding_boxes), desc="Processing Bounding Boxes") as pbar:
        # Submit each bounding box processing task to the executor
        future_to_bbox = {executor.submit(process_bbox, bbox): bbox for bbox in bounding_boxes}
        for future in concurrent.futures.as_completed(future_to_bbox):
            bbox = future_to_bbox[future]
            try:
                result = future.result()
                if result:
                    ocr_results.append(result)
            except Exception as e:
                print(f"Error processing bounding box: {e}")
            finally:
                pbar.update(1)

    # Extract the image name without path and file extension
    image_name = image_file.split('/')[-1]  # Get the file name
    image_name = image_name.rsplit('.', 1)[0]  # Remove file extension

    # Create a dictionary with the image name as the key and the list of OCR results as the value
    result_dict = {image_name: ocr_results}

    # Return the dictionary
    return result_dict

In [6]:
# image_folder = "/root/219_image_files"
# xml_folder = "/root/219_Final_xml"

# # Get lists of image and xml files in their respective folders
# image_files = os.listdir(image_folder)
# xml_files = os.listdir(xml_folder)

# # Create a set of xml file names for quick lookup
# xml_set = set(xml_files)

# # Initialize a list to hold the results
# results = []

# # Iterate through the image files
# for image_file in image_files:
#     # Check if the file has an image extension (jpg, jpeg, png)
#     if image_file.endswith('.jpg') or image_file.endswith('.jpeg') or image_file.endswith('.png'):
#         # Get the base name without the extension
#         base_name = os.path.splitext(image_file)[0]

#         # Create the expected xml file name
#         expected_xml_file = base_name + '.xml'

#         # Check if the corresponding xml file exists in the xml folder
#         if expected_xml_file in xml_set:
#             # Full paths for the image and xml files
#             image_file_path = os.path.join(image_folder, image_file)
#             xml_file_path = os.path.join(xml_folder, expected_xml_file)

#             # Call your existing functions with the image and XML file paths
#             response = google_cloud_vision_response(image_file_path)
#             p = get_response_inside_roi(response, image_file_path, xml_file_path)
#             #print(p)
#             # Create a dictionary with the base_name as the key and the list of strings as the value
#             #result_dict = {base_name: p}

#             # Add the result dictionary to the list
#             results.append(p)
#         else:
#             print(f"Corresponding XML file not found for image: {image_file}")

# # results
start_time = time.time()
print("Processing Starts .....")
image_folder = "/root/219_image_files"
xml_folder = "/root/219_Final_xml"

# Get lists of image and xml files in their respective folders
image_files = os.listdir(image_folder)
xml_files = os.listdir(xml_folder)

# Create a set of xml file names for quick lookup
xml_set = set(xml_files)

# Initialize a list to hold the results
results = []

# Iterate through the image files with an index
for index, image_file in enumerate(image_files):
    # Check if the file has an image extension (jpg, jpeg, png)
    if image_file.endswith('.jpg') or image_file.endswith('.jpeg') or image_file.endswith('.png'):
        # Get the base name without the extension
        base_name = os.path.splitext(image_file)[0]

        # Create the expected xml file name
        expected_xml_file = base_name + '.xml'

        # Check if the corresponding xml file exists in the xml folder
        if expected_xml_file in xml_set:
            # Full paths for the image and xml files
            image_file_path = os.path.join(image_folder, image_file)
            xml_file_path = os.path.join(xml_folder, expected_xml_file)

            # Log or print the index and file being processed
            print(f"Processing image {index}: {image_file}")

            # Call your existing functions with the image and XML file paths
            response = google_cloud_vision_response(image_file_path)
            p = get_response_inside_roi(response, image_file_path, xml_file_path)

            # Add the result to the results list
            results.append(p)
        else:
            print(f"Corresponding XML file not found for image {index}: {image_file}")
end_time = time.time()
print("Processing Ends .....")
elapsed_time = end_time - start_time
print("\n")
print(f"It took {elapsed_time} seconds to complete!")

Processing Starts .....
Processing image 0: 110318528_00003.jpg


Processing /root/219_image_files/110318528_00003.jpg: 87388.90000059882it [00:00, 89455.42it/s] 
Processing Bounding Boxes: 100%|██████████| 4/4 [00:00<00:00, 14.80it/s]


Processing image 1: 110318528_00004.jpg


Processing /root/219_image_files/110318528_00004.jpg: 86984.40000057528it [00:00, 97467.25it/s] 
Processing Bounding Boxes: 100%|██████████| 4/4 [00:00<00:00, 14.31it/s]


Processing image 2: 110318528_00005.jpg


...

Processing image 5263: 110978546_00098.jpg


Processing /root/219_image_files/110978546_00098.jpg: 85989.40000051736it [00:00, 100709.69it/s] 
Processing Bounding Boxes: 100%|██████████| 6/6 [00:00<00:00, 10.17it/s]


Processing image 5264: 110978546_00101.jpg


Processing /root/219_image_files/110978546_00101.jpg: 91682.70000084875it [00:00, 99176.84it/s] 
Processing Bounding Boxes: 100%|██████████| 6/6 [00:00<00:00, 10.13it/s]

Processing Ends .....


It took 5979.989311218262 seconds to complete!


In [7]:
def process_row(row, columns):
    image_id = row['Image']
    row_data = {col: str(row[col]) for col in columns}
    return image_id, row_data

def extract_data_to_dict(df, columns):
    """
    Extracts specified columns for each unique 'Image_ID' from a DataFrame, and
    stores them in a dictionary of dictionaries.

    :param df: DataFrame from which to extract data.
    :param columns: List of columns to be extracted.
    :return: Dictionary with 'Image_ID' as keys and corresponding data as values.
    """
    # Filter the dataframe to include only the specified columns and 'Image'
    columns_with_id = ['Image'] + columns
    #columns_with_id = ['Image_Id'] + columns
    filtered_df = df[columns_with_id].copy()  # Create a copy of the DataFrame slice

    # Replace NaN with empty strings
    filtered_df.fillna('', inplace=True)

    # Create a dictionary with 'Image' as keys and the other column data as values
    data_dict = {}

    # Process rows in parallel
    with concurrent.futures.ThreadPoolExecutor() as executor, tqdm(total=len(filtered_df), desc="Processing Rows") as pbar:
        # Submit each row processing task to the executor
        future_to_row = {executor.submit(process_row, row, columns): row for _, row in filtered_df.iterrows()}
        for future in concurrent.futures.as_completed(future_to_row):
            row = future_to_row[future]
            try:
                image_id, row_data = future.result()
                if image_id in data_dict:
                    # If the image_id already exists, append the new row data to it
                    if isinstance(data_dict[image_id], list):
                        data_dict[image_id].append(row_data)
                    else:
                        data_dict[image_id] = [data_dict[image_id], row_data]
                else:
                    # Create a new entry in the dictionary for the new image_id
                    data_dict[image_id] = row_data
            except Exception as e:
                print(f"Error processing row: {e}")
            finally:
                pbar.update(1)

    return data_dict

columns_to_extract = [
   'Event_Day_orig',
   'Event_Month_orig',
   'Event_Year_orig',
   'Given_Name_orig',
   'Surname_orig',
   'Sex_orig',
   'Age_orig',
   'Birth_Year_orig',
   'Death_Year_orig',
   'Father_Given_Name_orig',
   'Father_Surname_orig',
   'Mother_Given_Name_orig',
   'Mother_Surname_orig',
   'Spouse_Given_Name_orig',
   'Spouse_Surname_orig',
   'Spouse_Age_orig',
   'Spouse_Birth_Year_orig',
   'Spouse_Father_Given_Name_orig',
   'Spouse_Father_Surname_orig',
   'Spouse_Mother_Given_Name_orig',
   'Spouse_Mother_Surname_orig'
]


# Load the Excel file
file_path = "/root/keyed_data.xlsx"

df = pd.read_excel(file_path, dtype=str)  # Convert all columns to strings while reading
data_dict = extract_data_to_dict(df, columns_to_extract)

# Convert the dictionary to a JSON string with indentation for formatting
formatted_data_json = json.dumps(data_dict, indent=2, sort_keys=False)

Processing Rows: 100%|██████████| 23458/23458 [00:05<00:00, 4281.05it/s]


In [ ]:
# @title
# def map_and_combine_data(ocr_data, keyed_data):
#     combined_data = []
#     with concurrent.futures.ThreadPoolExecutor() as executor:
#         # Submitting each OCR entry processing task to the executor
#         futures = [executor.submit(process_ocr_entry, ocr_entry, keyed_data) for ocr_entry in ocr_data]
#         # Create tqdm progress bar
#         with tqdm(total=len(futures), desc="Processing OCR Entries") as pbar:
#             # Retrieving results as they complete
#             for future in concurrent.futures.as_completed(futures):
#                 combined_data.extend(future.result())
#                 pbar.update(1)
#     return combined_data

# def process_ocr_entry(ocr_entry, keyed_data):
#     combined_data = []
#     for image_id, ocr_text_list in ocr_entry.items():
#         key = image_id.split('.')[0]  # Removing file extension
#         if key in keyed_data:
#             json_data_list = keyed_data[key]
#             for i, ocr_text in enumerate(ocr_text_list):
#                 if i < len(json_data_list):
#                     try:
#                         # Ensure there is a corresponding JSON entry
#                         combined_entry = {
#                             'Input': ocr_text,
#                             'Output': json_data_list[i]  # Corresponding JSON entry for this OCR text
#                         }
#                         combined_data.append(combined_entry)
#                     except IndexError:
#                         # Handle the case where there is no JSON entry for the current index
#                         print(f"Warning: Insufficient JSON data for OCR text {ocr_text} in image {image_id}")
#                     except KeyError:
#                         # Handle the case where json_data_list is not a list
#                         print(f"Warning: Invalid JSON data for image {image_id}")
#                 else:
#                     # Skip this OCR text as there is no corresponding JSON data
#                     pass
#         else:
#             # Skip all OCR texts for this image as there is no keyed data
#             pass
#     return combined_data

In [8]:
def map_and_combine_data(ocr_data, keyed_data):
    combined_data = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Submitting each OCR entry processing task to the executor
        futures = [executor.submit(process_ocr_entry, ocr_entry, keyed_data) for ocr_entry in ocr_data]
        # Create tqdm progress bar
        with tqdm(total=len(futures), desc="Processing OCR Entries") as pbar:
            # Retrieving results as they complete
            for future in concurrent.futures.as_completed(futures):
                combined_data.extend(future.result())
                pbar.update(1)
    return combined_data

def process_ocr_entry(ocr_entry, keyed_data):
    combined_data = []
    for image_id, ocr_text_list in ocr_entry.items():
        key = image_id.split('.')[0]  # Removing file extension
        if key in keyed_data:
            json_data_list = keyed_data[key]
            # Ensure json_data_list is always a list
            if not isinstance(json_data_list, list):
                json_data_list = [json_data_list]
            for i, ocr_text in enumerate(ocr_text_list):
                if i < len(json_data_list):
                    try:
                        # Ensure there is a corresponding JSON entry
                        combined_entry = {
                            'Input': ocr_text,
                            'Output': json_data_list[i]  # Corresponding JSON entry for this OCR text
                        }
                        combined_data.append(combined_entry)
                    except IndexError:
                        # Handle the case where there is no JSON entry for the current index
                        print(f"Warning: Insufficient JSON data for OCR text {ocr_text} in image {image_id}")
                    except KeyError:
                        # Handle the case where json_data_list is not a list
                        print(f"Warning: Invalid JSON data for image {image_id}")
                else:
                    # Skip this OCR text as there is no corresponding JSON data
                    pass
        else:
            # Skip all OCR texts for this image as there is no keyed data
            pass
    return combined_data

In [9]:
keyed_data = json.loads(formatted_data_json)

# Call map_and_combine_data function with the dictionary
combined_data = map_and_combine_data(results, keyed_data)
# Continue with the rest of your code
df_combined = pd.DataFrame(combined_data)
df_combined.to_csv('final_formatted_dataset_NA0219_coordinates_rearranged.csv', index=False)

Processing OCR Entries: 100%|██████████| 5265/5265 [00:00<00:00, 208779.36it/s]


In [11]:
d = pd.read_csv("/root/final_formatted_dataset_NA0219_coordinates_rearranged.csv")
d.shape

(20727, 2)

In [11]:
h = pd.read_csv("/root/keyed_data.csv")
h.shape

(23458, 24)

In [12]:
type(keyed_data)

dict

In [16]:
len(combined_data)

20727

In [ ]:
type(results)

In [ ]:
len(results)

In [ ]:
results[0]

In [15]:
import pandas as pd
d = pd.read_excel("/root/keyed_data.xlsx")
d.shape

(23458, 24)